In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 為了處理方便，把 'train.csv' 和 'test.csv' 合併起來，'test.csv'的 Weather 欄位用 0 補起來。
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_test['Weather'] = np.zeros((len(df_test),))
city_coor = pd.read_csv("aus_loc_coor.csv", index_col = "loc")

# 以 train_end_idx 作為 'train.csv' 和 'test.csv' 分界列，
train_end_idx = len(df)
df = pd.concat([df, df_test], sort=False)

In [ ]:
cardinal = {
            'E':1+0j, 'ENE':0.92388+0.38268j, 'ESE':0.92388-0.38268j,
            'N':0+1j, 'NNE':0.38268+0.92388j, 'NNW':-0.38268+0.92388j,
            'S':0-1j, 'SSE':0.38268-0.92388j, 'SSW':-0.38268-0.92388j,
            'W':-1+0j, 'WNW':-0.92388+0.38268j, 'WSW':-0.92388-0.38268j,
            'SW':-0.7071-0.7071j, 'NE':0.7071+0.7071j, 'NW':-0.7071+0.7071j, 'SE':0.7071-0.7071j
        }

In [3]:
# Date Loc TempLow TempHigh Steaming Sun WindDir WindSpeed DayWindDir NightWindDir DayWindSpeed NightWindSpeed DayHumidity 
# NightHumidity DayPressure NightPressure DayCloud NightCloud DayTemp NightTemp Weather RISK_MM

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

# print(df.head(10))

labelencoder = LabelEncoder()
tmp = df['Loc'][df['Loc'].notnull()]
labelencoder.fit(tmp)

wind = MinMaxScaler()
tmp = df['WindSpeed'][df['WindSpeed'].notnull()]
wind.fit(tmp.values.reshape(-1,1))

print(df['Date'].values[4])
# df['Date'] = df['Date'].astype(str).str.split('/', n = 1).str.get(1)
print(df['Date'].values[4])
df['Date'] = labelencoder.fit_transform(df['Date']).astype(float)
print(df['Date'].values[4])
df['Loc'] = labelencoder.fit_transform(df['Loc']).astype(float)
df['WindDir'] = labelencoder.fit_transform(df['WindDir']).astype(float)
df['DayWindDir'] = labelencoder.fit_transform(df['DayWindDir']).astype(float)
df['NightWindDir'] = labelencoder.fit_transform(df['NightWindDir']).astype(float)
# df = df.drop(columns = [col for col in df.columns if df[col].dtype == np.object_])





nan
nan
3314.0


In [ ]:
# 將 missing value 補 0
for col in df.columns:
    if col not in ['Date', 'Loc', 'WindDir', 'DayWindDir', 'NightWindDir']:
        df[col] = df[col].fillna(df[col].median())
        # print(col)
        # print(df[col].dtype)
    else:
        df[col] = df[col].fillna(df[col].mean())
        # print(col)
        # print(df[col].dtype)

# train

In [4]:
from sklearn.model_selection import train_test_split

# X: det_lables, y: sol_lables
X = df.drop(columns = ['Weather']).values[:train_end_idx, :]
y = df['Weather'].values[:train_end_idx]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# print(X_train[0])

test = df.drop(columns = ['Weather']).values[train_end_idx:, :]

from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 42)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.preprocessing import normalize, StandardScaler

# X_train = normalize(X_train)
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)
test = sc.transform(test)

# train tree model
# model = GradientBoostingClassifier(max_depth=3, random_state=120)
# model = LogisticRegression(solver='newton-cholesky', max_iter=1000000, random_state=120)
# model = SVC(kernel='rbf', C=2, random_state=0, probability=True)
# model = XGBClassifier()
# model = LGBMClassifier()
model.fit(X_train, y_train)


# predict
y_pred_decision = model.predict(X_test)
print('Accuracy: %f' % accuracy_score(y_test, y_pred_decision))
print('f1-score: %f' % f1_score(y_test, y_pred_decision))


Accuracy: 0.897753
f1-score: 0.349321


In [7]:
ans_pred = model.predict(test)
df_sap = pd.DataFrame(ans_pred.astype(int), columns = ['Weather'])
df_sap.to_csv('myAns2.csv',  index_label = 'Id')